In [8]:
import librosa
import os
import numpy as np
import sys
from dtw import dtw
from numpy.linalg import norm
from numpy import array
import pyaudio
import wave

import heapq

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



**|Heap| = 50**  
**n_mfcc = 20**  
**DTWSeq = mfcc**  
**mfcc-MinMax**

In [15]:
def initialCorpus(path):
    # 音乐库位置
    audioList = os.listdir(path)

    raw_audioList = {}
    beat_database = {}

    for tmp in audioList:
        audioName = os.path.join(path, tmp)
        if audioName.endswith('.wav'):
            # 读入一维音频序列
            y, sr = librosa.load(audioName)
            # 提取 MFCC 特征
            f = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
            # 存入数据表
            beat_database[audioName] = f

    # 保存音乐节奏数据库
    np.save('beatDatabase_mfcc_20.npy', beat_database)
    
    return beat_database

In [16]:
def readCorpus(path):
    
    # 读入音乐节奏数据库
    all_data = np.load(path, allow_pickle=True)
    beat_database = all_data.item()
    
    return beat_database

In [17]:
def updateCorpus(path, dbpath):
    
    # 音乐库位置
    audioList = os.listdir(path)
    
    # 已保存序列的文件
    raw_db = readCorpus(dbPath)
    raw_files = raw_db.keys()
    
    for tmp in audioList:
        audioName = os.path.join(path, tmp)
        if audioName.endswith('.wav') and audioName not in raw_files:
            y, sr = librosa.load(audioName)
            # 提取 MFCC 特征
            f = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
            # 存入数据表
            beat_database[audioName] = f

    # 保存音乐节奏数据库
    np.save(dbpath, beat_database)

In [18]:
def normlize(data):
    n_mean = np.mean(data, axis=0)
    n_std  = np.std(data, axis=0)
    
    norm_data = np.divide(np.subtract(data, n_mean), n_std)
    return norm_data

In [19]:
def voiceCompare_quick(dbPath, tPath):
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()

    # 读入要识别的录音
    y, sr = librosa.load(tPath)

    # 识别录音的节奏序列
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    beat_frames = librosa.feature.delta(beat_frames,mode ='nearest')
    x = array(beat_frames).reshape(-1, 1)

    # 将待识别的录音序列与语料库中语音逐一做DTW对比
    compare_result = {}
    
    for songID in beat_database.keys():
        y = beat_database[songID]
        y = array(y).reshape(-1, 1)
        
        dist = dtw(x, y).distance
        # print('两段话的差异程度为： ', songID.split("\\")[1], ": ", dist)
        
        compare_result[songID] = dist

    matched_song = min(compare_result, key=compare_result.get)
    print("最接近的录音是：", matched_song)

In [20]:
from sklearn import preprocessing

def voiceCompare(dbPath, tPath):
    # ========= 最大检索数 =========
    aimNum = 50
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()

    # 读入要识别的录音
    y, sr = librosa.load(tPath)

    # ========= 提取录音的 MFCC 特征 =========
    x = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20).T  # n1 * 20
    lenx = len(x)
    
    # ========= 标准化 ===========
    for i in range(0, lenx):
        x[i] = preprocessing.minmax_scale(x[i])
        # x[i] = normlize(x[i])

    # == 将待识别的录音序列与语料库中语音逐一做DTW对比 ==
    
    # heap for [dist, 时间段，文件名]
    heap = []
    heapq.heapify(heap)  
    
    for songID in beat_database.keys():
        # 取出文件名对应的 mfcc 序列
        y = beat_database[songID].T
        
        leny = len(y) # n2 * 20 
        print(leny)
        
        # ========= 标准化 =========
        for i in range(0, leny):
            y[i] = preprocessing.minmax_scale(y[i])
            # y[i] = normlize(y[i])
        
        for tp in range(0, leny - lenx):
            # *加速* 设定距离上限
            full = False  # 堆是否已满
            dist_UB = -10000  # DTW 距离上限
            overBound = False  # 是否过限
            
            if (len(heap) >= aimNum):
                full = True
                dist_UB = -heap[0][0]  # heap top (biggest) DTW dist as UB  
                
            # 计算 DTW(y[tp : tp + lenx])
            total_dist = 0
            
            for i in range(0, lenx):
                # DTW dist
                total_dist += dtw(x[i], y[tp + i], distance_only=False).distance
                
                # *加速* 超过上限直接取消
                if (full and total_dist > dist_UB):
                    overBound = True
                    break
            
            # *加速* 超过上限
            if (overBound):
                continue
            
            # 入栈
            tupleY = (-total_dist, tp, songID) # dtw 距离加负数转为大根堆
            
            heapq.heappush(heap, tupleY)
            if (len(heap) > aimNum):
                heapq.heappop(heap)
            
            print(tupleY)
            
        # end for
        
        
    return heap

In [21]:
def getTimePoint_dense(dbPath, tPath, vheap):
    res_num = 20 # 定义取出前 res_num 位的结果作为识别结果
    
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()
    
    # 得到要识别的录音时长
    tTime = librosa.get_duration(filename=tPath)
    
    # 提取前 res_num 个相似的片段并输出对应时间段
    similar_n = heapq.nlargest(res_num, vheap)
    
    print("开始输出相似片段：")
    
    for i in range(0, res_num):
        music_name = similar_n[i][2]  # 录音文件名
        music_time = librosa.get_duration(filename=music_name)  # 录音时长
        
        music_pos = similar_n[i][1]  # 时间段所在帧数
        music_all = len(beat_database[music_name][0])  # 录音总帧数

        frag_st = music_time / music_all * music_pos  # 时间段起点
        frag_en = frag_st + tTime  # 时间段终点
        
        # print(music_name, music_time, music_pos, music_all, frag_st)
        # print("相似度第", i + 1, "位的为文件 ", music_name, "的 ", '%.2f' % frag_st, "到", '%.2f' % frag_en, "秒")
        
        print(music_name, ",", '%.2f' % frag_st, "秒,", '%.2f' % frag_en, "秒")


In [22]:
def getTimePoint(dbPath, tPath, vheap):
    # 读入语料库
    all_data = np.load(dbPath, allow_pickle=True)
    beat_database = all_data.item()
    
    # 得到要识别的录音时长
    tTime = librosa.get_duration(filename=tPath)
    
    heapq.nlargest(20, vheap)
    
    # ====== 对 vheap 进行去重 ======
    # 取出文件名
    name_set = set()
    for tp in vheap:
        name_set.add(tp[2])
    # print(name_set)
    
    # 合并下标差小于5的片段
    sheap = []
    for name in name_set:
        # 按下标排序
        nList = [x for x in vheap if x[2] == name]
        sortL = sorted(nList, key=lambda t:t[1])
        
        # 去重
        for tp in sortL:
            if len(sheap) < 1 or sheap[-1][2] != name or abs(sheap[-1][1] - tp[1]) > 5:
                sheap.append(tp)
            else:  
                if (sheap[-1][0] < tp[0]): 
                    sheap[-1] = tp  # 保留距离较小项

    # print(sheap)
    # 提取相似片段并输出对应时间段
    similar_n = sheap
    
    print("开始输出相似片段：")
    
    for i in range(0, len(sheap)):
        music_name = similar_n[i][2]  # 录音文件名
        music_time = librosa.get_duration(filename=music_name)  # 录音时长
        
        music_pos = similar_n[i][1]  # 时间段所在帧数
        music_all = len(beat_database[music_name][0])  # 录音总帧数
        
        frag_st = music_time / music_all * music_pos  # 时间段起点
        frag_en = frag_st + tTime  # 时间段终点
        # print(music_name, music_time, music_pos, "/", music_all, "=", frag_st)
        
        # print(music_name, music_time, music_pos, music_all, frag_st)
        # print("相似度第", i + 1, "位的为文件 ", music_name, "的 ", '%.2f' % frag_st, "到", '%.2f' % frag_en, "秒")
        
        print(music_name, ",", '%.2f' % frag_st, "秒,", '%.2f' % frag_en, "秒")

In [23]:
# 语料库路径
corpus_path = './corpus'

# 数据表路径
dbPath = './beatDatabase_mfcc_20.npy';

# test file path
# testPath = './input/00415250-前5s.wav'
testPath = './input/00429126-53s_60s.wav'
# testPath = './input/00430105-hou5s.wav'

In [24]:
# 1 初始化语料序列库
# beatDB = initialCorpus(corpus_path)

# 2 更新语料库中新音乐文件的序列
# updateCorpus(corpus_path, dbPath)

# 3 读入语料序列库
# beat_database = readCorpus(dbPath)

testPath = './input/00429126-53s_60s.wav'
vheap = voiceCompare(dbPath, testPath)

3876
(-1272.4287064820528, 0, './corpus\\00415250.wav')
(-1265.9078442007303, 1, './corpus\\00415250.wav')
(-1270.7586699575186, 2, './corpus\\00415250.wav')
(-1267.651317551732, 3, './corpus\\00415250.wav')
(-1258.4703255444765, 4, './corpus\\00415250.wav')
(-1254.2515805512667, 5, './corpus\\00415250.wav')
(-1260.9910843521357, 6, './corpus\\00415250.wav')
(-1265.9266250878572, 7, './corpus\\00415250.wav')
(-1270.022905960679, 8, './corpus\\00415250.wav')
(-1258.8563814908266, 9, './corpus\\00415250.wav')
(-1241.328565403819, 10, './corpus\\00415250.wav')
(-1214.4417929500341, 11, './corpus\\00415250.wav')
(-1188.7863148897886, 12, './corpus\\00415250.wav')
(-1180.6972980052233, 13, './corpus\\00415250.wav')
(-1172.671512171626, 14, './corpus\\00415250.wav')
(-1162.2643124312162, 15, './corpus\\00415250.wav')
(-1147.3719956427813, 16, './corpus\\00415250.wav')
(-1132.2109712809324, 17, './corpus\\00415250.wav')
(-1129.1676958948374, 18, './corpus\\00415250.wav')
(-1140.7793988138437,

(-1023.6977255344391, 1374, './corpus\\00415250.wav')
(-989.7761162221432, 1375, './corpus\\00415250.wav')
(-982.5541469454765, 1376, './corpus\\00415250.wav')
(-986.9752613008022, 1377, './corpus\\00415250.wav')
(-987.2861039638519, 1378, './corpus\\00415250.wav')
(-964.1061779260635, 1379, './corpus\\00415250.wav')
(-936.0793073773384, 1380, './corpus\\00415250.wav')
(-923.1497507989407, 1381, './corpus\\00415250.wav')
(-924.2201836109161, 1382, './corpus\\00415250.wav')
(-927.5212257057428, 1383, './corpus\\00415250.wav')
(-922.4596058726311, 1384, './corpus\\00415250.wav')
(-903.0822639763355, 1385, './corpus\\00415250.wav')
(-865.0098605453968, 1386, './corpus\\00415250.wav')
(-822.5172728896141, 1387, './corpus\\00415250.wav')
(-785.3981631398201, 1388, './corpus\\00415250.wav')
(-769.5972041785717, 1389, './corpus\\00415250.wav')
(-777.7690067589283, 1390, './corpus\\00415250.wav')
(-795.3662235736847, 1391, './corpus\\00415250.wav')
(-800.3838440775871, 1392, './corpus\\0041525

(-802.7779697179794, 2813, './corpus\\00429126.wav')
(-784.6229260414839, 5896, './corpus\\00429126.wav')
(-750.9945798963308, 5897, './corpus\\00429126.wav')
(-715.3528240919113, 5898, './corpus\\00429126.wav')
(-677.4923540949821, 5899, './corpus\\00429126.wav')
(-647.3456031084061, 5900, './corpus\\00429126.wav')
(-633.2066630721092, 5901, './corpus\\00429126.wav')
(-624.3513579368591, 5902, './corpus\\00429126.wav')
(-629.1539694815874, 5903, './corpus\\00429126.wav')
(-654.8446338027716, 5904, './corpus\\00429126.wav')
(-698.9722158163786, 5905, './corpus\\00429126.wav')
(-734.8540210276842, 5906, './corpus\\00429126.wav')
(-756.2362649291754, 5907, './corpus\\00429126.wav')
(-768.550636395812, 5908, './corpus\\00429126.wav')
(-768.0053540617228, 5909, './corpus\\00429126.wav')
(-776.3977795392275, 5910, './corpus\\00429126.wav')
(-789.130877956748, 5911, './corpus\\00429126.wav')
(-789.3720866888762, 6434, './corpus\\00429126.wav')
(-751.6037504225969, 6435, './corpus\\00429126.w

In [25]:
getTimePoint(dbPath, testPath, vheap)

开始输出相似片段：
./corpus\00429126.wav , 53.01 秒, 60.12 秒
./corpus\00429126.wav , 53.15 秒, 60.26 秒
./corpus\00429126.wav , 65.24 秒, 72.35 秒
./corpus\00429126.wav , 137.04 秒, 144.15 秒
./corpus\00429126.wav , 149.60 秒, 156.71 秒
./corpus\00430105.wav , 76.36 秒, 83.47 秒
./corpus\00430105.wav , 156.27 秒, 163.38 秒
./corpus\00429239.wav , 155.97 秒, 163.08 秒
./corpus\00429239.wav , 186.01 秒, 193.12 秒


In [ ]:
inPara = sys.argv

if (len(inPara) < 2):
    print("请输入待识别录音文件路径！")
else:
    if (len(inPara) > 2):
        print("给定语料库路径为:", sys.argv[2])
        corpus_path = sys.argv[2]
    else:
        print("默认语料库路径为：", corpus_path)
    
    if (len(inPara) > 3):
        print("给定数据表路径为:", sys.argv[3])
        dbPath = sys.argv[3]
    else:
        print("默认数据表路径为：", dbPath)
    
    testPath = sys.argv[1]
    vheap = voiceCompare(dbPath, testPath)
    getTimePoint(dbPath, testPath, vheap)
  

In [13]:
# beat_database = readCorpus(dbPath)
# len(beat_database['./corpus\\00415250.wav'])

In [26]:
testPath2 = './input/00430105-hou5s.wav'
vheap2 = voiceCompare(dbPath, testPath2)

3876
(-630.1832760572433, 0, './corpus\\00415250.wav')
(-626.6100344657898, 1, './corpus\\00415250.wav')
(-623.0175407528877, 2, './corpus\\00415250.wav')
(-619.8169311881065, 3, './corpus\\00415250.wav')
(-616.5797521471977, 4, './corpus\\00415250.wav')
(-613.9206947088242, 5, './corpus\\00415250.wav')
(-613.0515416264534, 6, './corpus\\00415250.wav')
(-613.1651991307735, 7, './corpus\\00415250.wav')
(-614.2743850648403, 8, './corpus\\00415250.wav')
(-616.7365712225437, 9, './corpus\\00415250.wav')
(-618.6829030513763, 10, './corpus\\00415250.wav')
(-618.498614192009, 11, './corpus\\00415250.wav')
(-617.1428053081036, 12, './corpus\\00415250.wav')
(-615.3171213269234, 13, './corpus\\00415250.wav')
(-612.7557292580605, 14, './corpus\\00415250.wav')
(-609.9931738972664, 15, './corpus\\00415250.wav')
(-605.8190496265888, 16, './corpus\\00415250.wav')
(-603.6354722678661, 17, './corpus\\00415250.wav')
(-601.8223753869534, 18, './corpus\\00415250.wav')
(-600.4091695249081, 19, './corpus\\0

(-349.95137499272823, 213, './corpus\\00415250.wav')
(-353.82002963125706, 214, './corpus\\00415250.wav')
(-361.5075783878565, 215, './corpus\\00415250.wav')
(-365.2415188997984, 216, './corpus\\00415250.wav')
(-367.28236581385136, 217, './corpus\\00415250.wav')
(-373.0193814486265, 218, './corpus\\00415250.wav')
(-377.70611496269703, 219, './corpus\\00415250.wav')
(-379.3143416196108, 220, './corpus\\00415250.wav')
(-381.3275844603777, 221, './corpus\\00415250.wav')
(-378.89212568104267, 222, './corpus\\00415250.wav')
(-381.30369049310684, 223, './corpus\\00415250.wav')
(-385.49607396125793, 224, './corpus\\00415250.wav')
(-386.0775819271803, 225, './corpus\\00415250.wav')
(-380.98753693699837, 226, './corpus\\00415250.wav')
(-373.08322699368, 227, './corpus\\00415250.wav')
(-363.36796145141125, 228, './corpus\\00415250.wav')
(-357.08178989589214, 229, './corpus\\00415250.wav')
(-358.7580218911171, 230, './corpus\\00415250.wav')
(-365.3675095587969, 231, './corpus\\00415250.wav')
(-37

(-309.59304408729076, 1474, './corpus\\00415250.wav')
(-307.73940193653107, 1475, './corpus\\00415250.wav')
(-306.4181883186102, 1476, './corpus\\00415250.wav')
(-302.1302408427, 1477, './corpus\\00415250.wav')
(-297.86628122627735, 1478, './corpus\\00415250.wav')
(-296.421967074275, 1479, './corpus\\00415250.wav')
(-299.47065638005733, 1480, './corpus\\00415250.wav')
(-303.62240685522556, 1481, './corpus\\00415250.wav')
(-304.8360493183136, 1482, './corpus\\00415250.wav')
(-303.07753236591816, 1483, './corpus\\00415250.wav')
(-296.55339546501637, 1484, './corpus\\00415250.wav')
(-288.2931173592806, 1485, './corpus\\00415250.wav')
(-279.67913027107716, 1486, './corpus\\00415250.wav')
(-274.20975448191166, 1487, './corpus\\00415250.wav')
(-272.06511457264423, 1488, './corpus\\00415250.wav')
(-273.8522604852915, 1489, './corpus\\00415250.wav')
(-273.6968798786402, 1490, './corpus\\00415250.wav')
(-270.23196391761303, 1491, './corpus\\00415250.wav')
(-264.6507730334997, 1492, './corpus\\0

(-253.89575181901455, 3260, './corpus\\00430105.wav')
(-247.12355233728886, 3261, './corpus\\00430105.wav')
(-237.7315415740013, 3262, './corpus\\00430105.wav')
(-232.21935212612152, 3263, './corpus\\00430105.wav')
(-233.383677855134, 3264, './corpus\\00430105.wav')
(-240.25416733324528, 3265, './corpus\\00430105.wav')
(-247.31465758383274, 3266, './corpus\\00430105.wav')
(-254.96781693398952, 3267, './corpus\\00430105.wav')
(-258.92377349734306, 3271, './corpus\\00430105.wav')
(-258.1690261512995, 3272, './corpus\\00430105.wav')


In [27]:
getTimePoint(dbPath, testPath, vheap2)

开始输出相似片段：
./corpus\00429126.wav , 84.84 秒, 91.95 秒
./corpus\00429126.wav , 160.35 秒, 167.46 秒
./corpus\00430105.wav , 75.57 秒, 82.68 秒
./corpus\00430105.wav , 75.76 秒, 82.87 秒
./corpus\00430105.wav , 75.97 秒, 83.08 秒
./corpus\00429239.wav , 84.36 秒, 91.47 秒
./corpus\00429239.wav , 109.62 秒, 116.73 秒
./corpus\00429239.wav , 109.76 秒, 116.87 秒
./corpus\00429239.wav , 144.29 秒, 151.40 秒
